<a href="https://colab.research.google.com/github/saurabhgoel1985/Ayundante/blob/master/MHA_and_PointWiseFeedforward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [2]:
!pip install transformers

In [12]:
!pip install tensorflow==2.0.0
import tensorflow as tf
print(tf.__version__)
tf.keras.backend.set_floatx('float64')

2.0.0


In [7]:
MHA = tf.random.uniform((2, 5, 12))
MHA

<tf.Tensor: id=13, shape=(2, 5, 12), dtype=float32, numpy=
array([[[0.7719016 , 0.355631  , 0.2772329 , 0.10835278, 0.6969818 ,
         0.00566828, 0.2011044 , 0.8636819 , 0.23043084, 0.31390703,
         0.08488917, 0.2594502 ],
        [0.14533627, 0.8333149 , 0.5274061 , 0.55183244, 0.5994328 ,
         0.9442836 , 0.69573927, 0.13309133, 0.03376734, 0.1574254 ,
         0.40020263, 0.36364424],
        [0.49100423, 0.7429167 , 0.45633197, 0.4332162 , 0.04702842,
         0.5555397 , 0.27800035, 0.5995772 , 0.2562729 , 0.19652367,
         0.686244  , 0.4573114 ],
        [0.42523098, 0.9743831 , 0.52106667, 0.48874712, 0.7798443 ,
         0.8804562 , 0.09797204, 0.9036846 , 0.8699672 , 0.17925704,
         0.7816372 , 0.7508968 ],
        [0.5485096 , 0.6083747 , 0.8659829 , 0.742537  , 0.42155766,
         0.20908976, 0.43249583, 0.03364968, 0.11123633, 0.18571615,
         0.13618445, 0.16790128]],

       [[0.62077534, 0.7207805 , 0.78099287, 0.09946251, 0.48591065,
         0

In [0]:
# https://www.tensorflow.org/tutorials/text/transformer

# MultiHeadAttention function from TF Transformer Tutorial

def split_heads(x, batch_size):
  """Split the last dimension into (num_heads, depth).
  Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
  """
  x = tf.reshape(x, (batch_size, -1, 4, 3))
  return tf.transpose(x, perm=[0, 2, 1, 3])

In [8]:
split_heads(MHA, 2)

<tf.Tensor: id=17, shape=(2, 4, 5, 3), dtype=float32, numpy=
array([[[[0.7719016 , 0.355631  , 0.2772329 ],
         [0.14533627, 0.8333149 , 0.5274061 ],
         [0.49100423, 0.7429167 , 0.45633197],
         [0.42523098, 0.9743831 , 0.52106667],
         [0.5485096 , 0.6083747 , 0.8659829 ]],

        [[0.10835278, 0.6969818 , 0.00566828],
         [0.55183244, 0.5994328 , 0.9442836 ],
         [0.4332162 , 0.04702842, 0.5555397 ],
         [0.48874712, 0.7798443 , 0.8804562 ],
         [0.742537  , 0.42155766, 0.20908976]],

        [[0.2011044 , 0.8636819 , 0.23043084],
         [0.69573927, 0.13309133, 0.03376734],
         [0.27800035, 0.5995772 , 0.2562729 ],
         [0.09797204, 0.9036846 , 0.8699672 ],
         [0.43249583, 0.03364968, 0.11123633]],

        [[0.31390703, 0.08488917, 0.2594502 ],
         [0.1574254 , 0.40020263, 0.36364424],
         [0.19652367, 0.686244  , 0.4573114 ],
         [0.17925704, 0.7816372 , 0.7508968 ],
         [0.18571615, 0.13618445, 0.1679

In [0]:
# https://www.tensorflow.org/tutorials/text/transformer

# Point wise FeedForward Network function from TF Transformer Tutorial

def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [10]:
# Creating a fixed tensor - Assuming single sentence with 4 words and each word having embedding of length 5
PWFFN = tf.convert_to_tensor(np.array([
    [[0.23449826, 0.4833721 , 0.11461055, 0.33866155, 0.7113544 ],
     [0.23449826, 0.4833721 , 0.11461055, 0.33866155, 0.7113544 ],

    [0.68879676, 0.40220594, 0.10274923, 0.9111701 , 0.6127192],
     [0.68879676, 0.40220594, 0.10274923, 0.9111701 , 0.6127192]]
    ]))
print(PWFFN)
PWFFN.shape

tf.Tensor(
[[[0.23449826 0.4833721  0.11461055 0.33866155 0.7113544 ]
  [0.23449826 0.4833721  0.11461055 0.33866155 0.7113544 ]
  [0.68879676 0.40220594 0.10274923 0.9111701  0.6127192 ]
  [0.68879676 0.40220594 0.10274923 0.9111701  0.6127192 ]]], shape=(1, 4, 5), dtype=float64)


TensorShape([1, 4, 5])

In [13]:
# Here d_model = 5 and assuming dff = 10 for this example
point_wise_feed_forward_network(5, 10)(PWFFN)

<tf.Tensor: id=166, shape=(1, 4, 5), dtype=float64, numpy=
array([[[-0.06158173, -0.00862907,  0.00598307,  0.17304289,
          0.24087195],
        [-0.06158173, -0.00862907,  0.00598307,  0.17304289,
          0.24087195],
        [ 0.03339511,  0.01389098,  0.36826271,  0.20109408,
          0.42988334],
        [ 0.03339511,  0.01389098,  0.36826271,  0.20109408,
          0.42988334]]])>